<h1>An overview of <code>AMRVW</code></h1>

<p>The <code>AMRVW</code> package implements some numerical linear algebra algorithms of Jared L. Aurentz, Thomas Mach, Leonardo Robol, Raf Vandebril, and David S. Watkins for finding eigenvalues of matrices through Francis's method.</p>

<p>An inspiration is to find  the roots of a polynomial through the eigenvalues of a companion matrix. This is implemented in <code>AMRVW</code> through the <code>roots</code> function.</p>

<p>To illustrate,</p>

In [1]:
using AMRVW
const A = AMRVW  # currently there are no exports
ps = [24.0, -50.0, 35.0, -10.0, 1.0]  #  (x-1)(x-2)(x-3)(x-4) = 24 -50x + 25x^2  -10x^3 +  x^4
A.roots(ps)

4-element Array{Complex{Float64},1}:
 0.9999999999999996 + 0.0im
 1.9999999999999951 + 0.0im
  3.000000000000039 + 0.0im
 3.9999999999999645 + 0.0im

<p>The example shows</p>

<ul>
<li><p>the interface expects polynomials specified through a vector of coefficients, <code>&#91;a0, a1, ..., an&#93;</code></p>
</li>
<li><p>the  4 roots, always as complex numbers</p>
</li>
<li><p>the fact that the roots are numeric approximations due to accumulated round off errors.</p>
</li>
</ul>

<p>Similarly, roots of polynomials over the  complex numbers can be found</p>

In [1]:
ps  =  [0.0 + 1.0im, -1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 - 1.0im, 1.0 + 0.0im] # (x-1)(x+1)(x-i)^2)(x+i)
A.roots(ps)

5-element Array{Complex{Float64},1}:
                   -1.0 + 1.040834085586084e-16im 
 -1.1299204313131748e-8 + 0.9999999959195923im    
 1.6653345369377348e-16 - 1.0000000000000002im    
  1.1299204904942824e-8 + 1.0000000040804067im    
     0.9999999999999999 + 3.3306690738754696e-16im

<p>There are other ways  to numerically find roots  of polynomials  in <code>Julia</code>, notably the <code>roots</code> function  of  the <code>Polynomials</code> package and the <code>roots</code> function of the <code>PolynomialRoots</code> package:</p>

<ul>
<li><p>unlike <code>Polynomials.roots</code> (but similar to <code>PolynomialRoots.roots</code>) this  <code>roots</code> function can work with  big floats and other floating point types.</p>
</li>
<li><p>For moderate sized polynomials ($n \approx 50$), <code>PolynomialRoots.roots</code> is faster than <code>Polynomials.roots</code> which is faster than <code>roots</code>, though all are fast. When $n \approx 75$, <code>roots</code> is faster (much so for large $n$ than <code>Polynomials.roots</code>).</p>
</li>
<li><p>Unlike both <code>Polynomials.roots</code> and <code>PolynomialRoots.roots</code> this <code>roots</code> function can accurately identify roots of polynomials of high degree.  (For a polynomial with $n$ random coefficients, e.g., <code>ps &#61; rand&#40;n&#41;</code>) <code>Polynomials.roots</code> will have troubles for n around 50; <code>PolynomialRoots.roots</code> will have issues for <code>n</code> around 300; <code>roots</code> can quickly handle degree 3000, and still be accurate for higher degrees.)</p>
</li>
<li><p>The <code>roots</code> function is shown by  the authors  to be  backward stable. The same isn't the case  for the other  two.</p>
</li>
<li><p>In the first example, the residual errors are  similar in size to <code>Polynomials.roots</code>, but  <code>PolynomialRoots.roots</code> the residual errors seem to be generally a bit smaller.</p>
</li>
</ul>

<h2>The companion matrix</h2>

<p>Both <code>roots</code> and <code>Polynomials.roots</code> use a companion matrix representation using the  eigenvalues of this matrix to identify the roots of the polynomial. (The  <code>PolynomialRoots.roots</code> function relies on a different method following a paper by <a href="https://arxiv.org/abs/1203.1034">Skowron and Gould</a>.</p>

<p>Using some functions within <code>AMRVW</code> we can see the companion matrix:</p>

In [1]:
ps = [24.0, -50.0, 35.0, -10.0, 1.0]  #  (x-1)(x-2)(x-3)(x-4) = 24 -50x + 25x^2  -10x^3 +  x^4
F = A.amrvw(ps)
M = Matrix(F) |> round2   # round2 is just M -> round.(M, digits=2)

4×4 Array{Float64,2}:
  0.0   0.0   0.0  24.0
 -1.0   0.0   0.0  50.0
  0.0  -1.0   0.0  35.0
  0.0   0.0  -1.0  10.0

In [1]:
using LinearAlgebra
eigvals(F)

4-element Array{Complex{Float64},1}:
 1.0000000000000002 + 0.0im
  1.999999999999983 + 0.0im
 3.0000000000000377 + 0.0im
 3.9999999999999845 + 0.0im

<p>The eigvenvalues of this matrix indeed are the roots of the polynomials. Internally, <code>amrvw</code> actually uses an enlarged matrix, with an extra dimension that is not shown here.</p>

<h2>Francis's Algorithm</h2>

<p>Francis's Algorithm begins with a QR decomposition <code>M</code>. For example,</p>

In [1]:
LinearAlgebra.qr(M)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
4×4 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
R factor:
4×4 Array{Float64,2}:
 -1.0   0.0   0.0  50.0
  0.0  -1.0   0.0  35.0
  0.0   0.0  -1.0  10.0
  0.0   0.0   0.0  24.0

<p>the decomposition in <code>AMRVW</code> is slightly different, though similar</p>

In [1]:
Matrix(F.QF)

4×4 Array{Float64,2}:
  0.0   0.0   0.0  1.0
 -1.0   0.0   0.0  0.0
  0.0  -1.0   0.0  0.0
  0.0   0.0  -1.0  0.0

In [1]:
Matrix(F.RF) |> round2

5×5 Array{Float64,2}:
 1.0  0.0  -0.0  -50.0   0.0
 0.0  1.0  -0.0  -35.0   0.0
 0.0  0.0   1.0  -10.0   0.0
 0.0  0.0   0.0   24.0  -1.0
 0.0  0.0   0.0    0.0   0.0

<p>(Here the <code>RF</code> matrix shows the extra size used internally in the algorithm.)</p>

<p>The idea of Francis's shifted algorithm is to identify shifts $\rho_1$, $\rho_2$, $\dots$, $\rho_m$ and generate a <em>unitary</em> matrix $V_0 = \alpha (A-\rho_1 I)(A-\rho_2 I)\cdots(A-\rho_m)I \cdot e_1$, $e_1$ being a unit vector with $1$ in the $1$ entry and $0$ elsewhere. As $V_0$ is unitary, the product $V_0^{-1}F V_0$ will have the same eigenvalues.  When $F$ is upper Hessenberg (upper triangular starting with the subdiagonal), as is the case with the companion matrix, then this product will be almost upper Hessenberg, save for a bulge.</p>

<p>In the real-coefficient case,  $m=2$ is used to allow the calculations to be done over the real numbers. For the complex-coefficient case, $m=1$ is possible.</p>

In [1]:
ps  =  [0.0 + 1.0im, -1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 - 1.0im, 1.0 + 0.0im]
F = A.amrvw(ps)
M = Matrix(F); n = size(M, 1)
MI = diagm(0 => ones(Complex{Float64}, 5)) # identity matrix
storage, ctr, m = A.make_storage(F), A.make_counter(F), 1
A.create_bulge(F.QF, F.RF, storage, ctr) # finds shifts and creates V_0
(V0 = storage.VU[1] * MI) |> round2

5×5 Array{Complex{Float64},2}:
 -0.7+0.08im  -0.71+0.0im   -0.0+0.0im  -0.0+0.0im  -0.0+0.0im
 0.71+0.0im    -0.7-0.08im   0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im     0.0+0.0im    1.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im     0.0+0.0im    0.0+0.0im   1.0+0.0im   0.0+0.0im
  0.0+0.0im     0.0+0.0im    0.0+0.0im   0.0+0.0im   1.0+0.0im

<p>Up to rounding, $V_0$ is unitary:</p>

In [1]:
isapprox(V0 * V0', MI, atol=1e-8)

true

<p>The matrix $V_0' M V_0$ has a bulge below the subdiagonal (the <code>&#91;3,1&#93;</code> position, illustrated with a <code>1</code> below):</p>

In [1]:
V0' * M * V0 |> round2 .|> !iszero

5×5 BitArray{2}:
 1  1  0  0  1
 1  1  0  0  1
 1  1  0  0  0
 0  0  1  0  0
 0  0  0  1  1

<p>The algorithm finds $V_1$ to chase the bulge downward:</p>

In [1]:
A.absorb_Ut(F.QF, F.RF, storage, ctr)
A.passthrough_triu(F.QF, F.RF, storage, ctr, Val(:right))
A.passthrough_Q(F.QF, F.RF, storage, ctr, Val(:right))
(V1 = storage.VU[1] * MI) |> round2

5×5 Array{Complex{Float64},2}:
 1.0+0.0im   0.0+0.0im     0.0+0.0im   0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.56-0.13im  -0.82+0.0im   0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.82+0.0im    0.56+0.13im  0.0+0.0im  0.0+0.0im
 0.0+0.0im   0.0+0.0im     0.0+0.0im   1.0+0.0im  0.0+0.0im
 0.0+0.0im   0.0+0.0im     0.0+0.0im   0.0+0.0im  1.0+0.0im

<p>And this produce will have a bulge in <code>&#91;4,2&#93;</code> position:</p>

In [1]:
V1' * (V0' * M * V0) * V1 |> round2 .|> !iszero

5×5 BitArray{2}:
 1  1  1  0  1
 1  1  1  0  1
 0  1  1  0  1
 0  1  1  0  0
 0  0  0  1  1

<p>And again:</p>

In [1]:
A.passthrough_triu(F.QF, F.RF, storage, ctr, Val(:right))
A.passthrough_Q(F.QF, F.RF, storage, ctr, Val(:right))
(V2 = storage.VU[1] * MI) |> round2

5×5 Array{Complex{Float64},2}:
  1.0+0.0im   0.0+0.0im    0.0+0.0im     0.0+0.0im    0.0+0.0im
  0.0+0.0im   1.0+0.0im    0.0+0.0im     0.0+0.0im    0.0+0.0im
 -0.0+0.0im  -0.0+0.0im  -0.47+0.17im  -0.87+0.0im   -0.0+0.0im
  0.0+0.0im   0.0+0.0im   0.87+0.0im   -0.47-0.17im   0.0+0.0im
  0.0+0.0im   0.0+0.0im    0.0+0.0im     0.0+0.0im    1.0+0.0im

In [1]:
V2' * (V1' * (V0' * M * V0) * V1) * V2 |> round2 .|> !iszero

5×5 BitArray{2}:
 1  1  1  1  1
 1  1  1  1  1
 0  1  1  1  1
 0  0  1  1  1
 0  0  1  1  1

<p>Once pushed to the bottom, the bulge is absorbed into the matrix, leaving an upper Hessenberg form.</p>

<p>If the shifts are appropriately chosen, after a few iterations this resulting matrix can have an eigenvalue immediately read off and after deflation subsequent eigenvalues can be found.</p>

<h3>Shifts</h3>

<p>Above the bulge is created with a single rotator. As mentioned, for the real variable case, two rotators are used, so that the computations can be kept using real numbers. In general, the AMRVW algorithm can be defined for $m$ rotators. These rotators are produced by <code>create_bulge</code>, as illustrated above, and stored in <code>storage.UV</code>.</p>

<p>The choice of shifts is <em>essentially</em> the eigenvalues of the lower $2 \times 2$ submatrix (In the $m$-shift case, the lower $m\times m$ submatrix). In the Vandrebril and Watkins paper it is mentioned that this choice <em>normally</em> yields quaratic convergence. That is, one of the rotators will become a diagonal rotator with <code>s</code> part $0$. When that happens, deflation can occur. The algorithm is applied on this smaller, deflated matrix, until the deflated matrix is comprised of no more than $m$ rotators, at least one of which is a diagonal rotator. At this point one or more eigenvalues can be found. This quadratic convergence implies that generally there are $\mathcal{O}(n)$ steps taken.</p>

<h2>The AMRVW decomposition of the companion matrix</h2>

<p>The main result of the two papers on "Fast and Backward Stable Computation of Roots of Polynomials" is a proof of backward stability of a method that utilizes a sparse factorization of both the <code>Q</code> and <code>R</code> parts of the QR decomposition of a companion matrix.</p>

<p>Returning to the real case, and digging into some structures, we can illustrate:</p>

In [1]:
ps = [24.0, -50.0, 35.0, -10.0, 1.0]
F = A.amrvw(ps)
F.QF.Q

AMRVW.DescendingChain{Float64,Float64,Array{AMRVW.Rotator{Float64,Float64},1}}(AMRVW.Rotator{Float64,Float64}[AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 1), AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 2), AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 3)])

<p>To explain, this is a "chain" of real rotators, more clearly seen with:</p>

In [1]:
Vector(F.QF.Q)

3-element Array{AMRVW.Rotator{Float64,Float64},1}:
 AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 1)
 AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 2)
 AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 3)

<p>A rotator is a matrix which is identical to the identity matrix except in the <code>&#91;i,i&#43;1&#93; × &#91;i, i&#43;1&#93;</code> block, in which case it takes the form of a rotator: <code>&#91;c s; -s c&#93;</code>. (Our rotators are in the different direction than those in the papers.) Here <code>c</code> and <code>s</code> are the cosine and sine of some angle. These rotators are indexed by <code>i</code> and we use the notation $U_i$ to indicate a rotator of this form for a given $i$. In the above, we  can see  with inspection that there are 3 rotators with $i$ being 1, 2, and 3. This set of rotators is "descending" due to their order (1 then 2 then 3); ascending would be 3 then 2 then 1. The product of descending rotators will be upper Hessenberg:</p>

In [1]:
Matrix(F.QF.Q)

4×4 Array{Float64,2}:
  0.0   0.0   0.0  1.0
 -1.0   0.0   0.0  0.0
  0.0  -1.0   0.0  0.0
  0.0   0.0  -1.0  0.0

<p>A rotator at level $i$ will commute with a rotator at level $j$ unless $|i-j| \leq 1$. In the case where $i-j = \pm 1$, a key computation is the "turnover", which represents $U_i V_j W_i$ as $VV_j WW_i UU_j$. With the turnover, we can easily pass a rotator through an ascending or descending chain without disturbing those patterns.</p>

<p>In the  above illustration of Francis's  algorithm, the matrices  $V_0$, $V_1$,  etc. can be seen to be  rotators of this type. More generally, a unitary matrix with $m$ shifts can be viewed as a product of $m$  such rotators.</p>

<p>The $R$ decomposition  is trickier.  In the initial QR decomposition, $R$ has a simple structure plus a rank one part (coming from the coefficients). The  decomposition has two chains, an ascending one, <code>Ct</code>, and a descending one, <code>B</code>:</p>

In [1]:
Ct = F.RF.Ct
B = F.RF.B

AMRVW.DescendingChain{Float64,Float64,Array{AMRVW.Rotator{Float64,Float64},1}}(AMRVW.Rotator{Float64,Float64}[AMRVW.Rotator{Float64,Float64}(-0.7536071065605803, 0.6573251318346122, 1), AMRVW.Rotator{Float64,Float64}(-0.8025327939615967, 0.5966080075025758, 2), AMRVW.Rotator{Float64,Float64}(-0.3843312210120439, 0.9231952732523013, 3), AMRVW.Rotator{Float64,Float64}(-0.04163054471218133, -0.999133073092352, 4)])

<p>These almost begin as inverses:</p>

In [1]:
MI = diagm(0 => ones(5))
(Z = Ct * (B * MI)) |> round2

5×5 Array{Float64,2}:
  1.0   0.0   0.0  0.0   0.0
 -0.0   1.0   0.0  0.0   0.0
 -0.0  -0.0   1.0  0.0   0.0
  0.0   0.0  -0.0  0.0  -1.0
  0.0   0.0   0.0  1.0   0.0

<p>However, <code>Ct</code> is cleverly chosen to encode the rank 1 part. This can be uncovered through the following:</p>

In [1]:
e1 = vcat(1, zeros(4))
en1 = vcat(zeros(4), 1)
rho = (en1' * (Ct * MI) * e1)

-0.015072142131211606

<p>and</p>

In [1]:
yt = -(1/rho * en1' * (Ct*(B*MI)))
Ct * (e1 * yt) |> round2

5×5 Array{Float64,2}:
  0.0   0.0  -0.0  -50.0   0.0
  0.0   0.0  -0.0  -35.0   0.0
  0.0   0.0  -0.0  -10.0   0.0
  0.0   0.0   0.0   24.0   0.0
 -0.0  -0.0  -0.0   -1.0  -0.0

<p>Leading to <code>R</code>:</p>

In [1]:
Z + Ct * (e1 * yt) |> round2

5×5 Array{Float64,2}:
 1.0  0.0  -0.0  -50.0   0.0
 0.0  1.0  -0.0  -35.0   0.0
 0.0  0.0   1.0  -10.0   0.0
 0.0  0.0   0.0   24.0  -1.0
 0.0  0.0   0.0    0.0   0.0

<p>The algorithm passes a rotator through this decomposition, which in turn relies on passing a rotator through the two chains <code>B</code> and <code>Ct</code>, which, with the turnover computation, is easily computed.</p>

<p>With these decompositions in mind, the computation above $V_0' M V_0$, can be seen as $U_1' Q R U_1$.  The product $U_1' Q = U_1' Q_1 Q_2 \cdots Q_k$ is just a product of two rotators at level 1, so $U_1' Q_1$ can be fused to give a new $\tilde{Q}_1$ in the descending chain factorization of $Q$.  The passthrough just mentioned allows $\tilde{Q} R U_1$ to have this form $\tilde{Q} \tilde{U}_1 \tilde{R}$ and by passing through the descending chain, we have this form $U_2 \hat{Q} \tilde{R}$. The matrix $V_1$ (of Francis's algorithm above) is seen to be $U_2$, as the similarity transform using $Q_1=U_2$ leaves the product $\hat{Q} \tilde{R} U_2$ having the same eigen values, but with the bulge shifted down one level. This basic idea forms the algorithm to chase the bulge. In the $m > 1$ case, some other details are included.</p>

<p>The decomposition of the companion matrix is sparse. Rather than require $O(n^2)$ storage, it only needs $O(n)$. The iterative algorithm is $O(n)$ per iteration  and  $O(n^2)$ overall, as compared to the $O(n^3)$ required in general. This reduction allows the method to be practical for large $n$, unlike <code>Polynomial.roots</code> which uses an $O(n^3)$ algorithm.</p>

<h3>Pencil decompositions</h3>

<p>In "<em>Fast and backward stable computation of roots of polynomials, Part II</em>" the method is extended to the pencil decomposition of a polynomial.  A pencil decomposition of a polynomial, is a specification where if $p = a_0 + a_1x^1 + \cdots + a_n x^n$ then $v_1 = a_0$, $v_{i+1} + w_i = a_i$, and $w_n = a_n$. This has some advantages in cases where the polynomial has a particularly small leading coefficient, since division by a tiny $a_n$ will result in very large entries. The algorithm uses two upper triangular matrices.</p>

<p>The <code>roots</code> function allows a pencil decomposition to be passed in as two vectors:</p>

In [1]:
ps = [24.0, -50.0, 35.0, -10.0, 1.0]
vs, ws = A.basic_pencil(ps)
A.roots(vs, ws)

4-element Array{Complex{Float64},1}:
 1.0000000000000009 + 0.0im
 1.9999999999999818 + 0.0im
  3.000000000000078 + 0.0im
 3.9999999999999325 + 0.0im

<h4>The Wilkinson polynomial</h4>

<p>The Wilkinson polynomial, $(x-1)(x-2)\cdots(x-20)$, poses a challenge for <code>roots</code>:</p>

In [1]:
import Polynomials
ps = Polynomials.coeffs(Polynomials.poly(1.0:20.0))
A.roots(ps)

20-element Array{Complex{Float64},1}:
 1.0000000000000577 + 0.0im               
 1.9999999930347512 + 0.0im               
  3.000029537272189 + 0.0im               
 3.9869525491878077 + 0.0im               
  4.786431607594509 - 0.5060566577457261im
  4.786431607594509 + 0.5060566577457261im
  5.433002217882397 - 1.6846138826413906im
  5.433002217882397 + 1.6846138826413906im
 6.2089383568863665 - 3.272153142653679im 
 6.2089383568863665 + 3.272153142653679im 
  7.293993085277627 - 5.50514106965025im  
  7.293993085277627 + 5.50514106965025im  
  9.232866337577452 - 8.824467844006536im 
  9.232866337577452 + 8.824467844006536im 
 13.960734522749238 - 13.641152708276774im
 13.960734522749238 + 13.641152708276774im
 14.278094051004537 + 0.0im               
 26.581678177676462 - 15.908676473031345im
 26.581678177676462 + 15.908676473031345im
  38.73669344722332 + 0.0im               

<p>The answer involves complex-valued roots, even though the roots are clearly integer valued. (The <code>Polynomials.roots</code> function and <code>PolynomialRoots.roots</code> do get  real answers for  this polynomial). As an aside, the exact implementation of the fundamental <code>turnover</code> operation will effect the number of such roots.  The issue of spurious complex-valued roots might be addressed by separating out the smaller coefficients from the large ones. Here is a function to split a polynomial into two pieces.</p>

In [1]:
function pencil_split(ps, n)
    vs = zeros(eltype(ps), length(ps)-1)
    ws = zeros(eltype(ps), length(ps)-1)

    vs[1:n] = ps[1:n]
    ws[n:end] = ps[n+1:end]

    vs, ws
end

pencil_split (generic function with 1 method)

<p>It turns out that with <code>n&#61;13</code> only real roots are identified:</p>

In [1]:
A.roots(pencil_split(ps, 13)...)

20-element Array{Complex{Float64},1}:
  -66.79268129903977 - 143.20307603125696im
  -66.79268129903977 + 143.20307603125696im
 -1.0426685512809595 - 23.057959641939608im
 -1.0426685512809595 + 23.057959641939608im
  1.0000000000000817 + 0.0im               
   2.000000002999921 + 0.0im               
  3.0000095616876536 + 0.0im               
   3.447485351069723 - 12.193013427214483im
   3.447485351069723 + 12.193013427214483im
   3.927411069770779 + 0.0im               
   4.271177500736296 - 0.5991868582165943im
   4.271177500736296 + 0.5991868582165943im
   4.522345460973428 - 7.536263155166513im 
   4.522345460973428 + 7.536263155166513im 
   4.542833282752819 - 1.6338560981164851im
   4.542833282752819 + 1.6338560981164851im
     4.7355528326321 - 2.9813715532559013im
     4.7355528326321 + 2.9813715532559013im
  4.7828186206955845 - 4.8187559005698475im
  4.7828186206955845 + 4.8187559005698475im

<p>The pencil here does a better job with this polynomial, but the choice of <code>13</code> was made with hindsight, not foresight.</p>

<h2>Other uses</h2>

<p>The <code>AMRVW.jl</code> package allows some other applications, though the exact interface needs to be settled on.</p>

<p>If we take rotators $Q_1, Q_2, \dots, Q_k$ their product will be upper Hessenberg:</p>

In [1]:
Qs = A.random_rotator.(Float64, [1,2,3,4])

4-element Array{AMRVW.Rotator{Float64,Float64},1}:
 AMRVW.Rotator{Float64,Float64}(0.45852386027933506, 0.8886820970147519, 1) 
 AMRVW.Rotator{Float64,Float64}(0.3187201870175313, 0.9478488499689757, 2)  
 AMRVW.Rotator{Float64,Float64}(0.018202161043929303, 0.9998343269428844, 3)
 AMRVW.Rotator{Float64,Float64}(0.5965388542979148, 0.8025841982701449, 4)  

In [1]:
MI = diagm(0 => ones(5))
(M = Qs * MI) |> round2

5×5 Array{Float64,2}:
  0.46   0.28   0.02   0.5   0.68
 -0.89   0.15   0.01   0.26  0.35
  0.0   -0.95   0.01   0.19  0.26
  0.0    0.0   -1.0    0.01  0.01
  0.0    0.0    0.0   -0.8   0.6 

<p>Their eigenvalues can be found:</p>

In [1]:
eigvals(M)

5-element Array{Complex{Float64},1}:
 -0.5201948410454089 - 0.8540476142169955im
 -0.5201948410454089 + 0.8540476142169955im
  0.6291264498176171 - 0.7773029719098482im
  0.6291264498176171 + 0.7773029719098482im
                 1.0 + 0.0im               

<p>But the sparse representation can be used to also find such eigenvalues:</p>

In [1]:
QF = A.q_factorization(A.DescendingChain(Qs))
F = A.QRFactorization(QF)  # defaults to identify R factorization
Matrix(F) |> round2 # same as M

5×5 Array{Float64,2}:
  0.46   0.28   0.02   0.5   0.68
 -0.89   0.15   0.01   0.26  0.35
  0.0   -0.95   0.01   0.19  0.26
  0.0    0.0   -1.0    0.01  0.01
  0.0    0.0    0.0   -0.8   0.6 

In [1]:
eigvals(F)

5-element Array{Complex{Float64},1}:
 -0.5201948410454089 - 0.8540476142169954im
 -0.5201948410454089 + 0.8540476142169954im
  0.6291264498176166 - 0.7773029719098479im
  0.6291264498176166 + 0.7773029719098479im
                 1.0 + 0.0im               

<p>The <code>qr_factorization</code> method can take a Hessenberg matrix and complete the factorization. For this case, we have:</p>

In [1]:
F = A.qr_factorization(M, unitary=true)
eigvals(F)

5-element Array{Complex{Float64},1}:
 -0.5201948410454088 - 0.8540476142169953im
 -0.5201948410454088 + 0.8540476142169953im
  0.6291264498176171 - 0.7773029719098475im
  0.6291264498176171 + 0.7773029719098475im
                 1.0 + 0.0im               

<p>When <code>unitary&#61;true</code> is the case, this will outperform <code>eigvals</code> once the matrix size is around 50 by 50 and is significantly more performant for the 150 by 150 case.</p>

<hr />

<p>Not all upper Hessenberg matrices can he expressed as a descending chain of rotators, as the latter is unitary. However, any upper Hessenberg matrix can easily be seen to be represented as a descending chain of rotators times an upper triangular matrix.</p>

<p>The Givens rotation is a rotator, $U$, chosen so that if $x= [a,b]$, then $Ux = [r,0]$. This allows, for example, the following:</p>

In [1]:
M = triu(rand(5,5), -1)  # upper Hessenberg

5×5 Array{Float64,2}:
 0.575239  0.723498  0.0696539  0.213845  0.431978
 0.258671  0.910522  0.20503    0.142894  0.449668
 0.0       0.581567  0.664934   0.181417  0.584563
 0.0       0.0       0.12919    0.943228  0.845225
 0.0       0.0       0.0        0.490341  0.146287

In [1]:
c,s,r = A.givensrot(M[1,1], M[2,1])
U1 = A.Rotator(c,s,1)
U1 * M |> round2

5×5 Array{Float64,2}:
 0.63  1.03  0.15  0.25  0.58
 0.0   0.53  0.16  0.04  0.23
 0.0   0.58  0.66  0.18  0.58
 0.0   0.0   0.13  0.94  0.85
 0.0   0.0   0.0   0.49  0.15

<p>That is the subdiagonal in column 1 is 0 Similarly, a <code>U2</code> could then be found so that subdiagonal in column 2 is 0, etc. That is a choice of rotators is available for $U_k U_{k-1} U_{k-2} \cdots U_2 U_1 M = R$. Setting $V_i = U_i'$, we have then $M = V_1 V_2 \cdots V_k R = QR$, where $Q$ is unitary and  in decomposed form, and $R$ is upper triangular.</p>

<p>For example:</p>

In [1]:
Us = Any[]
G = copy(M)
for i in 1:4
  c,s,r = A.givensrot(G[i,i], G[i+1,i])
  Ui =  A.Rotator(c,s,i)
  pushfirst!(Us, Ui)
  lmul!(Ui, G)
end
R = G
Qs = reverse(adjoint.(Us))

4-element Array{AMRVW.Rotator{Float64,Float64},1}:
 AMRVW.Rotator{Float64,Float64}(0.9120323890522732, -0.4101181796989774, 1) 
 AMRVW.Rotator{Float64,Float64}(0.6761397614905746, -0.7367733864163857, 2) 
 AMRVW.Rotator{Float64,Float64}(0.9322472668104619, -0.36182182566067456, 3)
 AMRVW.Rotator{Float64,Float64}(0.8652590654793266, -0.5013249939967509, 4) 

<p>With this, we can do the following:</p>

In [1]:
QF = A.q_factorization(A.DescendingChain(Qs))
RF = A.RFactorizationUpperTriangular(R)

F = A.QRFactorization(QF, RF)

Matrix(F)  - M |> round2# same as F

5×5 Array{Float64,2}:
 -0.0  -0.0  -0.0   0.0   0.0
 -0.0  -0.0   0.0  -0.0   0.0
  0.0   0.0   0.0   0.0  -0.0
  0.0   0.0   0.0   0.0   0.0
  0.0  -0.0  -0.0  -0.0   0.0

<p>And</p>

In [1]:
[eigvals(F) eigvals(M)]

5×2 Array{Complex{Float64},2}:
 -0.195973+0.0im  -0.195973+0.0im
  0.201442+0.0im   0.201442+0.0im
  0.579764+0.0im   0.579764+0.0im
   1.15121+0.0im    1.15121+0.0im
   1.50377+0.0im    1.50377+0.0im

<p>This patttern is encoded in the <code>qr_factorization</code> function, mentioned above. For any Hessenberg matrix it can be employed:</p>

In [1]:
H = hessenberg(rand(5,5))
F = A.qr_factorization(H.H)
eigvals(F)

5-element Array{Complex{Float64},1}:
   -0.4261221096964025 - 0.3773526516953606im
   -0.4261221096964025 + 0.3773526516953606im
 -0.008243763228993863 + 0.0im               
    0.2518255596727059 + 0.0im               
     2.526898476692753 + 0.0im               

<p>This approach is in the ballpark with <code>eigvals&#40;Matrix&#40;H.H&#41;&#41;</code>, though slower by a factor performance-wise, as the <code>R</code> part is not factored into rotators, so uses $\mathcal{O}(n)$ operations – not $\mathcal{O}(1)$ – in each step of the algorithm. (When <code>unitary&#61;true</code> is the case, the <code>R</code> part is much faster, so is competitive.</p>

<hr />

<p>The product of a descending chain of rotators is upper Hessenberg and the product of an  ascending chain or rotators is lower Hessenberg. With modifications, described in "A generalization of the multishift QR algorithm" a related algorithm can be employed. A "twisted chain" is one where a descending chain of rotators is permuted, an ascending chain being one possible twisting among many others.</p>

In [1]:
N = 4
T = Float64; S = T # real case here
MI = diagm(0 => ones(N+1))
Qs = A.random_rotator.(T, [4,3,2,1])
M = Qs * MI

QF = A.q_factorization(A.TwistedChain(Qs))
F = A.QRFactorization(QF)   # use default identify R factorization

[eigvals(M) eigvals(F)]

5×2 Array{Complex{Float64},2}:
 -0.0826325-0.99658im   -0.0826325-0.99658im 
 -0.0826325+0.99658im   -0.0826325+0.99658im 
    0.80553-0.592556im     0.80553-0.592556im
    0.80553+0.592556im     0.80553+0.592556im
        1.0+0.0im              1.0+0.0im     

<p>Here is another example with a CMV pattern to the twisted</p>

In [1]:
N = 5
MI = diagm(0 => ones(N+1))
Qs = A.random_rotator.(T, [1,3,5,2,4])
M = Qs * MI

QF = A.q_factorization(A.TwistedChain(Qs))

F = A.QRFactorization(QF)

[eigvals(M) eigvals(F)]

6×2 Array{Complex{Float64},2}:
 -0.0980614-0.99518im   -0.0980614-0.99518im 
 -0.0980614+0.99518im   -0.0980614+0.99518im 
   0.815051-0.579389im    0.815051-0.579389im
   0.815051+0.579389im    0.815051+0.579389im
   0.984803-0.173676im    0.984803-0.173676im
   0.984803+0.173676im    0.984803+0.173676im

<p>The implementation for twisted chains is not nearly as efficient as that for descending chains.</p>